In [ ]:
def GetCell(fileName):
    import requests
    import xml.etree.ElementTree as ET
    import pprint
    import numpy as np
    from copy import copy, deepcopy
    
    f = open(fileName,'r')
    text = f.read()
    root = ET.fromstring(text)
    components = root.findall('{http://www.cellml.org/cellml/1.1#}component')
    
    # Getting the names of all the sub-systems ==> subName
    subName=[]
    for c in components:
        subName.append(c.get('name'))
    
    #number of models in the file 
    modelNum=len(components)
    #number of variables in each model 
    compNum=len(components[0])
    
    
    variables=[]; els=[]; init=[]
    
    for comp in components:
        variables.append(comp.findall('{http://www.cellml.org/cellml/1.1#}variable'))
 
    for var in variables:
        for v in var:
            els.append(v.attrib['name'])
            if 'initial_value' in v.attrib: # if any initial value exists take it
                init.append(v.attrib['initial_value'])    

    # Separating the groups of elements in each model
    j=0
    elements=[]
    for i in range(0,modelNum*compNum,compNum):    
        elements.append(els[i:i+compNum].copy())
        j+=1
    # Separating the groups of initial values in each model
    j=0
    init_vals=[]
    for i in range(0,modelNum*compNum,compNum):    
        init_vals.append(init[i:i+compNum].copy())
        j+=1
        
    return [modelNum,subName, elements, init_vals]

In [ ]:
def BG_Components(modelNum, elements,init_vals):
    
    import pprint
    import numpy as np
    import BondGraphTools as bgt
    from copy import copy, deepcopy
    
    # assigning the value of each element to its equivalent component value in bond graphs
    x0_value=[]; x1_value=[]; x2_value=[]; x3_value=[]; x4_value=[]; x5_value=[];
    R_value=[]; R1_value=[]; R2_value=[];
    L_value=[]; L1_value=[]; L2_value=[]; C_value=[]; C1_value=[]; C2_value=[]; 
    
    for m in range(modelNum):
        for k in range(len(elements[0])):
            if elements[m][k]=='q':
                x0_value.append(init_vals[m][k])
            elif elements[m][k]=='q_1':
                x1_value.append(init_vals[m][k])
            elif elements[m][k]=='q_2':
                x2_value.append(init_vals[m][k])
            elif elements[m][k]=='p':
                x3_value.append(init_vals[m][k])
            elif elements[m][k]=='p_1':
                x4_value.append(init_vals[m][k])
            elif elements[m][k]=='p_2':
                x5_value.append(init_vals[m][k])
            elif elements[m][k]=='R':
                R_value.append(init_vals[m][k])
            elif elements[m][k]=='R1':
                R1_value.append(init_vals[m][k])
            elif elements[m][k]=='R2':
                R2_value.append(init_vals[m][k])
            elif elements[m][k]=='L':
                L_value.append(init_vals[m][k])
            elif elements[m][k]=='L1':
                L1_value.append(init_vals[m][k])
            elif elements[m][k]=='L2':
                L2_value.append(init_vals[m][k])
            elif elements[m][k]=='C':
                C_value.append(init_vals[m][k])
            elif elements[m][k]=='C1':
                C1_value.append(init_vals[m][k])
            elif elements[m][k]=='C2':
                C2_value.append(init_vals[m][k])

    # Here by each iteration we create the BG components of each model ...
    # by adding the model number at the end of each component name, indicating that ...
    # each component belongs to which model
    
    C=[]; C1=[]; C2=[]; R=[]; R1=[]; R2=[]; L=[]; L1=[]; L2=[];
    zero_junc_a=[]; zero_junc_b=[]; zero_junc_c=[]; one_junc_a=[]; one_junc_b=[]; one_junc_c=[];
    
    for m in range(modelNum):
        C.append(bgt.new("C", value=float(C_value[m]), name='C_'+str(m)))
        C1.append(bgt.new("C", value=float(C1_value[m]), name='C1_'+str(m)))
        C2.append(bgt.new("C", value=float(C2_value[m]), name='C2_'+str(m)))
        R.append(bgt.new("R", value=float(R_value[m]), name='R_'+str(m)))
        R1.append(bgt.new("R", value=float(R1_value[m]), name='R1_'+str(m)))
        R2.append(bgt.new("R", value=float(R2_value[m]), name='R2_'+str(m)))
        L.append(bgt.new("I", value=float(L_value[m]), name='L_'+str(m)))
        L1.append(bgt.new("I", value=float(L1_value[m]), name='L1_'+str(m)))
        L2.append(bgt.new("I", value=float(L2_value[m]), name='L2_'+str(m)))
        
        zero_junc_a.append(bgt.new("0", name='zero_junc_a_'+str(m)))
        zero_junc_b.append(bgt.new("0", name='zero_junc_b_'+str(m)))
        zero_junc_c.append(bgt.new("0", name='zero_junc_c_'+str(m)))
        one_junc_a.append(bgt.new("1", name='one_junc_a_'+str(m)))
        one_junc_b.append(bgt.new("1", name='one_junc_b_'+str(m)))
        one_junc_c.append(bgt.new("1", name='one_junc_c_'+str(m)))
        
    return [C,C1,C2,R,R1,R2,L,L1,L2,zero_junc_a,zero_junc_b,zero_junc_c,one_junc_a,one_junc_b,one_junc_c,x0_value,x1_value,x2_value,x3_value,x4_value,x5_value]


In [ ]:
def CBR_sorting(C,C1,C2,R,R1,R2,L,L1,L2,zero_junc_a,zero_junc_b,zero_junc_c,one_junc_a,one_junc_b,one_junc_c):
    # By the following appending, all the same components in each model are put together
    # So we need to separate them and gather all the components of each model in a row
    CBR=[]
    CBR.append([one_junc_a,zero_junc_a,one_junc_b,zero_junc_b,one_junc_c,zero_junc_c,R,R1,R2,C,C1,C2,L,L1,L2])  
    
    connectivity_bonds_row=[]
    for j in range(len(CBR[0][0])):
        for i in range(len(CBR[0])):
            connectivity_bonds_row.append(CBR[0][i][j])
            
    connectivity_bonds_column=connectivity_bonds_row
            
    return [connectivity_bonds_column,connectivity_bonds_row]        

In [ ]:
def Index_C(modelNum,subName,*con):    
    
    connection=list(con)
    conIter=int(len(connection)/4)
    connecting_models=[]
    ports=[]
    ConMod=[]
   # Dividing the elements in the input connections into lists with 4 elements, so each list represents ...
   # a separate connection between the modules
    for p in range(0,len(connection),4):
        ConMod.append(connection[p:p+4])
    
    # For the given inputs in connections, we assign numbers so the calculations can go on ...
    # For instance we assign {0,1,2} for {C,C1,C2} respectively and we assign {0,1,2,...} for the model components
    for m in range(conIter):
        for i in range(0,4):
            for j in range(len(subName)):
                if ConMod[m][i]==subName[j]:
                    connecting_models.append(j)
                    if ConMod[m][i+1]=='C':
                        ports.append(0)
                    if ConMod[m][i+1]=='C1':
                        ports.append(1)
                    if ConMod[m][i+1]=='C2':
                        ports.append(2)

    
    # Now we split the connection modules and ports two by two to specify the detailed connections:
    Con_Mod=[]
    Port_Mod=[]
    for p in range(0,len(connecting_models),2):
        Con_Mod.append(connecting_models[p:p+2])
        Port_Mod.append(ports[p:p+2])

    
    
    return [Con_Mod,Port_Mod]

In [ ]:
def CM_Creation(modelNum):
    import pprint
    import numpy as np
    from copy import copy, deepcopy
    
     #  connectivity matrix for one sample branching vessel
    connectivity_matrix_BV=[[0,1,0,0,0,0,1,0,0,0,0,0,1,0,0],
                             [0,0,1,0,1,0,0,0,0,1,0,0,0,0,0],
                             [0,0,0,1,0,0,0,1,0,0,0,0,0,1,0],
                             [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0],
                             [0,0,0,0,0,1,0,0,1,0,0,0,0,0,1],                           
                             [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
                             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]


    CM={}
    
    # Creating a dictionary with empty lists for adding the connectivity matrices for each sub-system
    for n in range(modelNum):
        CM[str(n)]=deepcopy(connectivity_matrix_BV)
    

    Length=0
    for n in range(modelNum):
        Length+=len(CM[str(n)])
    connectivity_matrix_0=np.zeros((Length,Length))  

    
    k=0
    for n in range(modelNum):
        for i in range(k,k+len(CM[str(n)])):
            for j in range(k,k+len(CM[str(n)])):
                connectivity_matrix_0[i][j]=deepcopy(CM[str(n)][i-k][j-k])
        k+=len(CM[str(n)])
        
    return connectivity_matrix_0

In [ ]:
def Whole_CM(Con_Mod,Port_Mod,connectivity_bonds_row,connectivity_matrix_0):
# We want to connect the created sub-systems together by inserting "1"s in the connectivity matrix
    from copy import copy, deepcopy
    
    Name=[]
    for i in range(len(Con_Mod)):
        for j in range(2):
            m=Con_Mod[i][j]
            p=Port_Mod[i][j]
            if p == 0:
                Name.append('one_junc_a_'+str(m))
            elif p == 1:
                Name.append('zero_junc_b_'+str(m))
            elif p == 2:
                Name.append('zero_junc_c_'+str(m))

           
    Connection_between_models=[]
    for name in Name:
        for index in range(len(connectivity_bonds_row)):
            if name == connectivity_bonds_row[index].name:
                Connection_between_models.append(index)
                break
 
    connectivity_matrix=deepcopy(connectivity_matrix_0)
    for i in range(0,len(Connection_between_models),2):
        II=Connection_between_models[i]
        JJ=Connection_between_models[i+1]
        connectivity_matrix[II][JJ]=1
        
    return connectivity_matrix

In [ ]:
def BondsConnection(connectivity_matrix,connectivity_bonds_row,connectivity_bonds_column):
    import numpy as np
    import BondGraphTools as bgt
# Creating the bonds between the components based on the elements of the connectivity matrix
    Length=len(connectivity_matrix)
    bonds=[]  
    for i in range(Length):
        for j in range(Length):
            if connectivity_matrix[i][j]==1:
                bonds.append((connectivity_bonds_row[i],connectivity_bonds_column[j]))
            else:
                m=0
    
    
    # connecting the components for a bond graph representation of the system
    for head, tail in bonds:
        bgt.connect(head,tail)
    
    return bonds

In [ ]:
# Combining the initial values of the state variables
def combineDict(*dicts):
    incKey = 0
    newDict = {}
    for dictionary in dicts:
        for val in dictionary:
            newDict['x_'+str(incKey)] = val 
            incKey+=1
    return newDict



In [ ]:
def Get_Boundary(modelNum,subName,bonds,model):
    import BondGraphTools as bgt

    count_one_junc_a={}
    count_zero_junc_b={}
    count_zero_junc_c={}


    for m in range(0,modelNum):
        itr_a=0
        itr_b=0
        itr_c=0
        for i in range(0,len(bonds)):
            for j in range(0,2):
                if bonds[i][j].name=='one_junc_a_'+str(m):
                    itr_a+=1
                    count_one_junc_a[str(m)]=itr_a
                elif bonds[i][j].name=='zero_junc_b_'+str(m):
                    itr_b+=1
                    count_zero_junc_b[str(m)]=itr_b
                elif bonds[i][j].name=='zero_junc_c_'+str(m):
                    itr_c+=1
                    count_zero_junc_c[str(m)]=itr_c

    boundary_names = ('Se', 'Sf1', 'Sf2')                
    boundary_values = {}
    Se=[]; Sf1=[]; Sf2=[];

    for m in range(0,modelNum):
        if count_one_junc_a[str(m)] != 4:
            boundary_values[boundary_names[0]+'_'+str(m)+'_value']=input('Enter value for {} in model {}: '.format(boundary_names[0],subName[m]))
            Se.append(bgt.new("Se", value=float(boundary_values[boundary_names[0]+'_'+str(m)+'_value']), name='Se_'+str(m)))
        if count_zero_junc_b[str(m)] != 3:
            boundary_values[boundary_names[1]+'_'+str(m)+'_value']=input('Enter value for {} in model {}: '.format(boundary_names[1],subName[m]))
            Sf1.append(bgt.new("Sf", value=float(boundary_values[boundary_names[1]+'_'+str(m)+'_value']), name='Sf1_'+str(m)))
        if count_zero_junc_c[str(m)] != 3:
            boundary_values[boundary_names[2]+'_'+str(m)+'_value']=input('Enter value for {} in model {}: '.format(boundary_names[2],subName[m]))
            Sf2.append(bgt.new("Sf", value=float(boundary_values[boundary_names[2]+'_'+str(m)+'_value']), name='Sf2_'+str(m)))
    bgt.add(model,Se,Sf1,Sf2)   
    
    for index in range(len(model.components)):
        for m in range(modelNum):
            if 'Se_'+str(m) == model.components[index].name:
                for j in range(len(model.components)):
                    if 'one_junc_a_'+str(m) == model.components[j].name:
                        bgt.connect(model.components[index],model.components[j])

    for index in range(len(model.components)):
        for m in range(modelNum):                    
            if 'Sf1_'+str(m) == model.components[index].name:
                for j in range(len(model.components)):
                    if 'zero_junc_b_'+str(m) == model.components[j].name:
                        bgt.connect(model.components[j],model.components[index])

    for index in range(len(model.components)):
        for m in range(modelNum):                    
            if 'Sf2_'+str(m) == model.components[index].name:
                for j in range(len(model.components)):
                    if 'zero_junc_c_'+str(m) == model.components[j].name:
                        bgt.connect(model.components[j],model.components[index])

    return model